<a href="https://colab.research.google.com/github/srishmaalladi/Katha-vachak/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
from unsloth import FastLanguageModel
model,tokenizer=FastLanguageModel.from_pretrained('/content/drive/MyDrive/final_model')

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
from transformers import TextStreamer
import re

# Define story-related keywords
story_keywords = ["story", "generate", "tale", "narrative", "adventure"]

def contains_story_keywords(user_input):
    # Check for any story-related keywords in the user input
    return any(keyword in user_input.lower() for keyword in story_keywords)

# Sample user input
user_input = "fuck off"  # Replace this with the actual user input

# Check if the input contains story-related keywords
if contains_story_keywords(user_input):
    # User input contains story-related keywords; generate the story
    alpaca_prompt = "Generate a story about {}."  # Modify the prompt as needed

    # Prepare inputs for the model
    inputs = tokenizer(
        [
            alpaca_prompt.format(user_input)  # Fill in the user input
        ], return_tensors="pt").to("cuda")

    # Enable native 2x faster inference using FastLanguageModel
    FastLanguageModel.for_inference(model)  # This line is essential for performance

    # Create a TextStreamer for displaying the output
    text_streamer = TextStreamer(tokenizer)

    # Generate the story
    _ = model.generate(**inputs, streamer=text_streamer)
else:
    # User input does not contain story-related keywords; provide an alternative response
    print("I am trained to generate stories. Sorry, I cannot help with that.")


I am trained to generate stories. Sorry, I cannot help with that.
